In [41]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer
import nltk
import json
import pandas as pd
from datasets import *
import numpy as np
import torch

nltk.download('punkt')
SEED = 42

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
split = "test" # seocify which split to evaluate
# split = "val"
# split = "train"
model_path = "../../models"
data_path_pairs = '../../data/processed/prompt_reply_pairs.csv'

In [43]:
model_name = "t5-sl-small-finetuned-assistant"
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}/{model_name}")
model = AutoModelForSeq2SeqLM.from_pretrained(f"{model_path}/{model_name}").to("cuda")

In [44]:
data = pd.read_csv(data_path_pairs, sep=";")
data

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
47509,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
47510,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
47511,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
47512,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test


In [45]:
data[data["split"] == split][:10]

,prompt,reply,rank,split
42639,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test
42640,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test
42641,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test
42642,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test
42643,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test
42644,Kakšen je geometrijski pomen izpeljanke?,Vrednost odvoda (realne) funkcije (realnega ar...,1,test
42645,Vzemimo naključno matriko $n\krat n$ z realnim...,"Verjetnost, da imajo lastne vrednosti naključn...",1,test
42646,"Recimo, da je samovozeči avto v situaciji, ko ...",Avtonomni avtomobili morajo biti programirani ...,0,test
42647,"Recimo, da je samovozeči avto v situaciji, ko ...","Avto mora biti programiran tako, da zagotavlja...",1,test
42648,"Recimo, da je samovozeči avto v situaciji, ko ...","To vrsto etičnih dilem, kot je tista, ki jo pr...",2,test


In [46]:
eval_data = Dataset.from_pandas(data[data['split'] == split][["prompt", "reply"]])
eval_data

Dataset({
    features: ['prompt', 'reply', '__index_level_0__'],
    num_rows: 4875
})

In [47]:
eval_data[:10]["prompt"]

['Vzemimo naključno matriko $n\\krat n$ z realnimi elementi, od katerih je vsak element izbran neodvisno od drugih iz standardne normalne porazdelitve. Kako oceniti verjetnost, da imajo vse lastne vrednosti te matrike pozitiven realni del? Kakšno je asimptotično obnašanje te verjetnosti, ko $n$ teži k neskončnosti?',
 'Hvala vam! Jasno mi je',
 'Hvala vam! Jasno mi je',
 'Hvala vam! Jasno mi je',
 'Kakšen je geometrijski pomen izpeljanke?',
 'Kakšen je geometrijski pomen izpeljanke?',
 'Vzemimo naključno matriko $n\\krat n$ z realnimi elementi, od katerih je vsak element izbran neodvisno od drugih iz standardne normalne porazdelitve. Kako oceniti verjetnost, da imajo vse lastne vrednosti te matrike pozitiven realni del? Kakšno je asimptotično obnašanje te verjetnosti, ko $n$ teži k neskončnosti?',
 'Recimo, da je samovozeči avto v situaciji, ko sta dve možnosti: povoziti starca in rešiti otroka ali povoziti otroka, da bi rešil starca, kako naj bo avto programiran?',
 'Recimo, da je sam

In [48]:
def convert_to_features(examples):
    prefix_in = "Uporabnik: "
    # prefix_in = ""
    examples["prompt"] = [prefix_in + prompt for prompt in examples["prompt"]]
    # prefix_out = "Asistent: "
    prefix_out = ""
    examples["reply"] = [prefix_out + reply for reply in examples["reply"]]
    
    model_inputs = tokenizer(examples['prompt'], pad_to_max_length=True, max_length=512, truncation=True, return_tensors='pt')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reply'], pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [49]:
eval_data = eval_data.map(convert_to_features, batched=True, load_from_cache_file=False)
eval_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_data

Map:   0%|          | 0/4875 [00:00<?, ? examples/s]c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.wa

Dataset({
    features: ['prompt', 'reply', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4875
})

### Evaluate

#### Qualitative

The generation ran almost 30minutes

In [60]:
# split so it fits on a 12gb gpu - couldn't find a better way (model.generate() options that would do this automatically...)
indices = np.concatenate([np.arange(0, eval_data.num_rows, eval_data.num_rows // 100), [eval_data.num_rows]])

# outputs = torch.tensor([], dtype=torch.int64).to("cuda")
outputs = []
for i, _ in enumerate(indices[:-1]):
    input_ids = eval_data[indices[i]:indices[i+1]]['input_ids'].to("cuda")
    # outputs = torch.cat((outputs, model.generate(test_data[indices[i]:indices[i+1]]['input_ids'].to("cuda"), max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=1)), dim=0) # problem with different tensor sizes
    outputs.append(model.generate(eval_data[indices[i]:indices[i+1]]['input_ids'].to("cuda"), max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=1))

In [61]:
# results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = []
for output in outputs:
    results.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

In [62]:
# just append the results and save
result_data = pd.read_csv(data_path_pairs, sep=";")
result_data = result_data[result_data['split'] == split].reset_index(drop=True)
result_data["generated"] = results
result_data.to_csv(f"../../data/t5_results/prompt_reply_pairs_generated_{model_name}.csv", sep=";", index=False)
result_data

,prompt,reply,rank,split,generated
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,"Verjetnost, da imajo vse lastne vrednosti matr..."
1,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test,Ni za kaj!
2,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test,Ni za kaj!
3,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test,Ni za kaj!
4,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test,"Da, geometrijski pomen izpeljanke je matematič..."
...,...,...,...,...,...
4870,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test,"Seveda, tukaj je nekaj elektronskih setov bobn..."
4871,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test,"Seveda, tukaj je nekaj elektronskih setov bobn..."
4872,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ..."
4873,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test,"Seveda, tukaj je nekaj možnih rešitev, ki jih ..."


In [63]:
# remove the duplicate prompts (we generate the same reply but we have them because during training they have different replies)
# also only keep the prompt and generated columns
result_data = result_data.drop_duplicates(subset=["prompt"])[["prompt", "generated"]]
result_data.to_csv(f"../../data/t5_results/prompt_reply_pairs_generated_{model_name}_no_duplicates.csv", sep=";", index=False)
result_data

,prompt,generated
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Verjetnost, da imajo vse lastne vrednosti matr..."
1,Hvala vam! Jasno mi je,Ni za kaj!
4,Kakšen je geometrijski pomen izpeljanke?,"Da, geometrijski pomen izpeljanke je matematič..."
7,"Recimo, da je samovozeči avto v situaciji, ko ...","Seveda, tukaj je primer, kako programirati sam..."
10,"Želim pripraviti pico s feferoni, mi lahko nav...","Tukaj je nekaj korakov, ki jih lahko naredite ..."
...,...,...
4863,Mi lahko poveste več informacij o Franciscu Mo...,Francisco Morazán je bil francoski fizik in ma...
4866,"Mi lahko poveste, kateri so glavni rezultati a...","Seveda, tukaj je nekaj glavnih rezultatov ali ..."
4868,"Imam elektronski set bobnov, a vsakič, ko igra...","Seveda, tukaj je nekaj nasvetov, ki jih lahko ..."
4870,"Hvala, bom videl, če lahko naredim te sprememb...","Seveda, tukaj je nekaj elektronskih setov bobn..."


In [64]:
input = "Uporabnik: Kdo je France Prešeren?"
input = tokenizer(input, return_tensors="pt").to("cuda")
outputs = model.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu v različnih delih sveta.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju glasbe, glasbe in glasbe.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju umetnosti in umetnosti.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju glasbe, glasbe in glasbe. Znan je tudi po svojih delih, kot so pesmi, eseji, pesmi in pesmi.']

#### Quantitative

In [50]:
rouge = load_metric("rouge")
bert = load_metric("bertscore")

In [51]:
def compute_rouge(decoded_preds, decoded_labels):
    # predictions, labels = eval_pred
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [62]:
labels = eval_data["labels"]
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
len(decoded_labels)

4875

In [63]:
# load the results obtained earlier to not have to generate them again as it takes a long time
result_data = pd.read_csv(f"../../data/t5_results/prompt_reply_pairs_generated_{model_name}.csv", sep=";")
decoded_preds = result_data["generated"].to_list()
len(decoded_preds)

4875

In [64]:
quantitative_data = pd.DataFrame({"decoded_preds": decoded_preds, "decoded_labels": decoded_labels})
quantitative_data

,decoded_preds,decoded_labels
0,"Verjetnost, da imajo vse lastne vrednosti matr...","Ocenjevanje natančne verjetnosti, da imajo vse..."
1,Ni za kaj!,Vedno z veseljem pomagamo! Vas zanima še kaj?
2,Ni za kaj!,prosim! V veselje mi je bilo pomagati vam. Ved...
3,Ni za kaj!,Z veseljem pomagam! Ali potrebujete še kaj?
4,"Da, geometrijski pomen izpeljanke je matematič...","Geometrični pomen odvoda je, da pokaže, kako h..."
...,...,...
4870,"Seveda, tukaj je nekaj elektronskih setov bobn...",Elektronski seti bobnov so na splošno tišji od...
4871,"Seveda, tukaj je nekaj elektronskih setov bobn...","To je odvisno od seta bobnov, ki ga trenutno u..."
4872,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ...",Pod komplet lahko poskusite postaviti izolacij...
4873,"Seveda, tukaj je nekaj možnih rešitev, ki jih ...","Da, obstaja več načinov, kako to popraviti. Z ..."


Compute with the duplicates (different replies for duplicate prompts druing training)

In [65]:
result_rouge = compute_rouge(decoded_preds, decoded_labels)
print(result_rouge)

{'rouge1': 18.8125, 'rouge2': 4.5543, 'rougeL': 12.713, 'rougeLsum': 16.8139}


In [70]:
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

{'Precision': 0.6645311312003014, 'Recall': 0.6427384702853668, 'F1': 0.6519272175385402}


Compute without duplicates

In [71]:
quantitative_data = quantitative_data.drop_duplicates(subset=["decoded_preds"])
display(quantitative_data)
decoded_preds = quantitative_data["decoded_preds"].to_list()
decoded_labels = quantitative_data["decoded_labels"].to_list()

,decoded_preds,decoded_labels
0,"Verjetnost, da imajo vse lastne vrednosti matr...","Ocenjevanje natančne verjetnosti, da imajo vse..."
1,Ni za kaj!,Vedno z veseljem pomagamo! Vas zanima še kaj?
4,"Da, geometrijski pomen izpeljanke je matematič...","Geometrični pomen odvoda je, da pokaže, kako h..."
7,"Seveda, tukaj je primer, kako programirati sam...",Avtonomni avtomobili morajo biti programirani ...
10,"Tukaj je nekaj korakov, ki jih lahko naredite ...","jasno! Začnemo lahko s seznamom sestavin, ki j..."
...,...,...
4863,Francisco Morazán je bil francoski fizik in ma...,Seveda! Francisco Morazán je bil honduraški vo...
4866,"Seveda, tukaj je nekaj glavnih rezultatov ali ...",Poskušal je spremeniti Srednjo Ameriko v velik...
4868,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ...","Obstaja več stvari, s katerimi lahko poskusite..."
4870,"Seveda, tukaj je nekaj elektronskih setov bobn...",Elektronski seti bobnov so na splošno tišji od...


In [72]:
result_rouge = compute_rouge(decoded_preds, decoded_labels)
print(result_rouge)

{'rouge1': 20.118, 'rouge2': 4.9437, 'rougeL': 13.3134, 'rougeLsum': 18.0039}


In [73]:
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

{'Precision': 0.6720401199131198, 'Recall': 0.6401641544793714, 'F1': 0.6545916620613571}
